In [ ]:
import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import joblib

SEMILLA_ALEATORIEDAD = 123
np.random.seed(SEMILLA_ALEATORIEDAD)

In [ ]:
from pandas import set_option
#from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Crea un gráfico circular con el porcentaje de fotógrafos prefesionales y los que no
def show_loan_distrib(y):
    i=0
    normales = 0
    fotografos = 0
    while (i<len(y)):
        if y[i]==0:
            normales+=1
        else: fotografos+=1
        i+=1

    tamanos = [fotografos, normales]
    etiquetas = [f'Fotógrafos profesionales - {tamanos[0]}', f'Otros usuarios - {tamanos[1]}']

    plt.pie(tamanos, labels=etiquetas, autopct = '%1.1f%%', explode = [0, 0.1])
    plt.show()

In [ ]:
# Función para representar la matriz de confusión de las etiquetas

from sklearn.metrics import confusion_matrix

def matrix_confusion_etiq(etiq_manual, etiq_auto):
    cm = confusion_matrix(etiq_manual, etiq_auto) 
    print(cm)
    
    # Para mostrar la información en porcetanjes también
    cm_porcentaje = cm.astype('float')/cm.sum()*100

    fig, ax = plt.subplots()
    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    for (i, j), val in np.ndenumerate(cm):
        percentage = cm_porcentaje[i, j]
        if i==0 and j==0:
            ax.text(j, i, f'{val} ({percentage:.2f}%)', ha='center', va='center', color = 'white')
        else:
            ax.text(j, i, f'{val} ({percentage:.2f}%)', ha='center', va='center', color = 'black')


    labels = ['No Fotógrafo', 'Fotógrafo']

    ax.set_xlabel('Etiqueta Automática')
    ax.set_ylabel('Etiqueta Manual')
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    ax.set_title('Matriz de Confusión de etiquetados')

    plt.show() 

In [ ]:
# Función para representar la matriz de confusión de las predicciones

from sklearn.metrics import confusion_matrix

def matrix_confusion_pred(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred) 
    print(cm)
    
    # Para mostrar la información en porcetanjes también
    #cm_porcentaje = cm.astype('float')/cm.sum()*100

    fig, ax = plt.subplots()
    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    for (i, j), val in np.ndenumerate(cm):
        #percentage = cm_porcentaje[i, j]
        fontsize = 12
        color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax.text(j, i, f'{val}', ha='center', va='center', color=color, fontsize=fontsize)

    labels = ['No Fotógrafo', 'Fotógrafo']

    ax.set_xlabel('Predicción')
    ax.set_ylabel('Ground-truth')
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    ax.set_title('Matriz de confusión de la predicción de expertos')

    plt.show() 

In [ ]:
# Función que plotea la curva ROC

from sklearn.metrics import roc_curve, auc

def roc_auc(y_test, y_pred_test_proba):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_test_proba)
    roc_auc = auc(fpr, tpr)

    #plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random guess (AUC = 0.5)')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()

In [ ]:
# Función que plotea la curva Precision-Recall

from sklearn.metrics import precision_recall_curve, auc
from numpy import argmax

def pr_auc(y_test, y_pred_test_proba):
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_test_proba)
    
    # convert to f score
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = argmax(fscore)
    
    pr_auc = auc(recall, precision)
    # locate the index of the largest f score
    ix = argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
    # plot the roc curve for the model
    no_skill = len(y_test[y_test==1]) / len(y_test)
    plt.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
    plt.plot(recall, precision, marker='.', label= f'PR curve (area = {pr_auc:.2f})')
    plt.scatter(recall[ix], precision[ix], marker='o', color='black', label='Best')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    # show the plot
    plt.show()
    return thresholds[ix]

In [ ]:
# Función que representa en un diagrama de barras las medidas de desempeño en la predicción

# valores es el array con los valores de las métricas que queremos plotear
# ponemos el título del plot como argumento para especificar el caso que estamos ploteando
def show_bars_metrics(valores, titulo):
    nombres = ["Accuracy","Precision","Recall","f1", "ROC-AUC", "AP/PR-AUC"]
    color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

    # Nombres y valores de las medidas
    #valores = fila_max_manual.values[2:7]

    # Crear una figura y un eje
    fig, ax = plt.subplots()

    # Crear la gráfica de barras
    bars = ax.bar(nombres, valores, color=color, width=0.65)

    # Agregar etiquetas de valores a las barras
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width() / 2, height), 
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=16)

    # Agregar título y etiquetas de ejes
    ax.set_title(titulo, fontweight='bold', fontsize=20)
    ax.set_xlabel('Medidas', fontsize=12)
    ax.set_ylabel('Valores', fontsize=12)
    plt.yticks(fontsize=14)

    # Mostrar la gráfica
    plt.show()

In [ ]:
# Función que dibuja las distribuciones de las características individuales, para cada tipo de etiquetado

# str_etiq es el string del nombre que tiene la variable de clase en el data set df
def show_individual_distrib(df, str_etiq):
    df_prof = df[df[str_etiq] == 1]
    df_noProf = df[df[str_etiq] == 0]

    # Quitamos la columna de respuesta
    df_dist = df_manual.iloc[:,:-1]

    # Calculamos el número de filas y columnas necesarias para organizar los subtrazados
    n_rows = (len(df_dist.columns) + 3) // 4  # Aproximadamente 4 subplots por fila
    n_cols = min(len(df_dist.columns), 4)

    # Ajustamos el tamaño de la figura
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 4 * n_rows))

    # Ploteamos la distribución de cada columna en un subplot separado
    for i, column in enumerate(df_dist.columns):
        ax = axes[i // n_cols, i % n_cols]  # Calculamos la posición del subplot en la cuadrícula
        # Cada diagrama tendrá 40 barras como máximo
        df_prof[column].hist(bins=40, color = 'orange', alpha = 0.5, ax=ax)
        df_noProf[column].hist(bins=40, color = 'blue', alpha = 0.5,ax=ax)
        ax.set_title('{}'.format(column))
        ax.set_xlabel('Valor')
        ax.set_ylabel('Frecuencia')

    # Ajusta automáticamente el diseño de la figura para evitar superposiciones
    plt.tight_layout()
    plt.show()

In [ ]:
# Nos devuelve la información sobre la asimetría y la curtosis

# Nos devuelve el skewness y la kurtosis de cada característica
from scipy.stats import skew, kurtosis

def info_asymetry(df):
    skewness = df.apply(skew)
    kurt = df.apply(kurtosis)

    result = pd.DataFrame({
        'Feature': df.columns,
        'Skewness': skewness,
        'Kurtosis': kurt
    }).reset_index(drop=True)
    return result

In [ ]:
# Dibuja la matriz de correlación de Pearson entre los atributos

def show_correlation(df):
    correlation_matrix=df.corr().abs()
    sns.set(font_scale = 0.8)

    f, ax = plt.subplots(figsize=(12,12))

    sns.heatmap(correlation_matrix, cmap='YlGnBu', square=True, ax = ax)

    # Rotar las etiquetas del eje x
    plt.xticks(rotation=90)

    f.tight_layout()

In [ ]:
# Partición de los datos

def data_partition(df, proportion, str_etiq):
    train_df, test_df = train_test_split(df, test_size=proportion, random_state=SEMILLA_ALEATORIEDAD, stratify=df[str_etiq])

    # Separamos la variable de clase de las demás
    train_manual_x = train_df.loc[:, train_df.columns != str_etiq]
    train_manual_y = train_df.loc[:, str_etiq]

    test_manual_x = test_df.loc[:, test_df.columns != str_etiq]
    test_manual_y = test_df.loc[:, str_etiq]
    
    return [train_manual_x, train_manual_y, test_manual_x, test_manual_y]

In [ ]:
# Escalado con StandardScaler

from sklearn.preprocessing import StandardScaler

# Escalado estándar
def std_Scaler_data(train_x, test_x):
    
    scaler = StandardScaler()
    scaler.fit(train_x)
    trainScaled_x = pd.DataFrame(scaler.transform(train_x), columns=train_x.columns, index=train_x.index)
    testScaled_x = pd.DataFrame(scaler.transform(test_x), columns=test_x.columns, index=test_x.index)
    return [trainScaled_x, testScaled_x]

In [ ]:
# Escalado con QuantileTransformer

from sklearn.preprocessing import QuantileTransformer

# QuantileTransformer
def qt_Transformer_data(train_x, test_x):
    
    qt = QuantileTransformer(output_distribution='normal')
    qt.fit(train_x)
    trainQT_x = pd.DataFrame(qt.transform(train_x), columns=train_x.columns, index=train_x.index)
    testQT_x = pd.DataFrame(qt.transform(test_x), columns=test_x.columns, index=test_x.index)
    return [trainQT_x, testQT_x]

In [ ]:
# Clase que representa el transformador donde se elimina del conjunto las columnas con valores constantes

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class ConstantValueThresholdEliminator(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.9):
        self.threshold = threshold
        self.columns_to_drop = []

    def fit(self, X, y=None):
        # Calcular el umbral de valores constantes para cada columna
        for column in X.columns:
            most_frequent_value_count = X[column].value_counts().max()
            if most_frequent_value_count / len(X) > self.threshold:
                self.columns_to_drop.append(column)
        return self

    def transform(self, X, y=None):
        # Eliminar las columnas que exceden el umbral de valores constantes
        return X.drop(columns=self.columns_to_drop)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [ ]:
# Función que hace uso de la clase ConstantValueThresholdEliminator para eliminar columnas constantes

def constant_elimination(train_x, test_x, threshold):
    transformer_cte = ConstantValueThresholdEliminator(threshold=threshold)
    transformer_cte.fit(train_x)
    train_cte_x = transformer_cte.transform(train_x)
    test_cte_x = transformer_cte.transform(test_x)
    return [train_cte_x, test_cte_x]

In [ ]:
# Clase que representa el transformador donde se elimina del conjunto las columnas más correlacionadas

from sklearn.base import BaseEstimator, TransformerMixin

class CorrelationThresholdTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.features_to_keep_ = None
        self.to_remove = None

    def fit(self, X, y=None):
        # Convertimos a DataFrame si no lo es
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        # Calculamos la matriz de correlación
        corr_matrix = X.corr().abs()

        # Calculamos la correlación media de cada variable con el resto
        mean_correlation = corr_matrix.mean(axis=0)

        # Identificamos las variables a eliminar basadas en el umbral
        self.to_remove = set()
        for i in range(len(corr_matrix.columns)):
            for j in range(i + 1, len(corr_matrix.columns)):
                if corr_matrix.iloc[i, j] > self.threshold:
                    if mean_correlation.iloc[i] > mean_correlation.iloc[j]:
                        self.to_remove.add(corr_matrix.columns[i])
                    else:
                        self.to_remove.add(corr_matrix.columns[j])

        # Guardamos las variables a mantener
        self.features_to_keep_ = [col for col in X.columns if col not in self.to_remove]
        return self
    def get_removed(self):
        return self.to_remove

    def transform(self, X, y=None):
        # Convertimos a DataFrame si no lo es
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        # Retornamos el dataset con las variables seleccionadas
        return X[self.features_to_keep_]

    def get_support(self):
        # Método para obtener las características seleccionadas
        return self.features_to_keep_

In [ ]:
# Función que usa CorrelationThresholdTransformer para eliminar columnas correlacionadas

def corr_elimination(train_x, test_x, threshold):
    transformer_corr = CorrelationThresholdTransformer(threshold=threshold)
    transformer_corr.fit(train_x)
    train_corr_x = transformer_corr.transform(train_x)
    test_corr_x = transformer_corr.transform(test_x)
    #print(transformer_corr.get_removed())
    return [train_corr_x, test_corr_x]

In [ ]:
# Función que usa un entrenamiento de Random Forest para seleccionar atributos poco relevantes en el entrenamiento

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

def RelevantInformationEliminator(X, y, threshold=0.01):
    clf = RandomForestClassifier(n_estimators=250,
                              random_state=SEMILLA_ALEATORIEDAD)

    clf.fit(X, y)

    # Plot feature importance
    feature_importance = clf.feature_importances_
    # make importances relative to max importance
    #feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    
    features_eliminated = []
    for index in sorted_idx:
        if feature_importance[index] < threshold:
            features_eliminated.append(X.columns[index])

    return features_eliminated

In [ ]:
# Función que elimina los atributos poco relevantes usando la función RelevantInformationEliminator

def low_relevance_elimination(train_x, train_y, test_x, threshold):
    # Seleccionamos las columnas poco relevantes
    features_eliminated = RelevantInformationEliminator(train_x, train_y, threshold)
    print(features_eliminated)
    train_rel_x = train_x.drop(columns = features_eliminated)
    test_rel_x = test_x.drop(columns = features_eliminated)
    return [train_rel_x, test_rel_x]

In [ ]:
# Función que aplica PCA al conjunto de datos

from sklearn.decomposition import PCA

def dataset_pca(train_x, test_x, var_explicated):
    pca = PCA(n_components = var_explicated)
    train_pca_x = pca.fit_transform(train_x)
    test_pca_x = pca.transform(test_x)

    train_pca_x = pd.DataFrame(train_pca_x)
    test_pca_x = pd.DataFrame(test_pca_x)
    
    return [train_pca_x, test_pca_x]

In [ ]:
# Función que aplica SMOTE al conjunto de datos

from imblearn.over_sampling import SMOTE

def dataset_smote(train_x, train_y, with_sampling_strategy, k_neighbors):
    if with_sampling_strategy and k_neighbors != 0:
        oversample = SMOTE(random_state=SEMILLA_ALEATORIEDAD, sampling_strategy=0.5, k_neighbors=k_neighbors)
    elif with_sampling_strategy:
        oversample = SMOTE(random_state=SEMILLA_ALEATORIEDAD, sampling_strategy=0.5)
    else:
        oversample = SMOTE(random_state=SEMILLA_ALEATORIEDAD)
        
    smote_x, smote_y = oversample.fit_resample(train_x, train_y)
    return [smote_x, smote_y]

In [ ]:
# Función que aplica el bloque de preprocesamiento al conjunto de datos

def dataset_bloque(train_x, train_y, test_x, threshold_cte, threshold_corr,
                  threshold_relevance):
    train_cte_x, test_cte_x = constant_elimination(train_x, test_x, threshold_cte)
    
    train_corr_cte_x, test_corr_cte_x = corr_elimination(train_cte_x, test_cte_x, threshold_corr)
    
    train_bloque_x, test_bloque_x = low_relevance_elimination(train_corr_cte_x, train_y, test_corr_cte_x, threshold_relevance)
    
    return [train_bloque_x, test_bloque_x]

In [ ]:
# Función que crea la lista de conjuntos de datos con preprocesamientos distintos

def conjuntos_preprocesamiento(train_x, test_x, train_y, test_y,
                              with_smote=False, sampling_strategy = False, k_neighbors = 0):
    conjuntos = []
    
    trainScaled_x, testScaled_x = std_Scaler_data(train_x, test_x)
    conjuntos.append(["std", trainScaled_x, train_y, testScaled_x, test_y])
    
    
    trainQT_x, testQT_x = qt_Transformer_data(train_x, test_x)
    conjuntos.append(["qt", trainQT_x, train_y, testQT_x, test_y])
    
    if with_smote:
        smote_std_x, smote_std_y = dataset_smote(trainScaled_x, train_y, sampling_strategy, k_neighbors)
        conjuntos.append(["std_smote", smote_std_x, smote_std_y, testScaled_x, test_y])
        
        smote_qt_x, smote_qt_y = dataset_smote(trainQT_x, train_y, sampling_strategy, k_neighbors)
        conjuntos.append(["qt_smote", smote_qt_x, smote_qt_y, testQT_x, test_y])
        
    
    return conjuntos

In [ ]:
# Función que establece los algoritmos de aprendizaje supervisado a usar

# Spot-Check Algorithms
def ImbGetBasedModelDef():
    basedModels = []
    # Lineales (estoy pensando en usar LR en lugar de esta)
    basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
    # No lineales
    basedModels.append(('SVM'  , SVC(probability=True, class_weight='balanced')))
    # Ensamblados
    basedModels.append(('RF'   , RandomForestClassifier(class_weight='balanced')))
    basedModels.append(('GBM'  , GradientBoostingClassifier()))
    
    return basedModels

In [ ]:
# Función que aplica el bucle general de selección de modelos a partir de una búsqueda de hiperparámetros, para cada combinación
# de algoritmo y preprocesado

from sklearn import metrics

# El scoring por defecto es roc-auc, pero en clases desbalanceadas deberemos poner "average_precision"
def bucle_proceso_general(conjuntos,  grids, scoring = "roc_auc"):
    resultados = pd.DataFrame(columns = ["modelo", "conjunto", "accuracy", "precision", "recall", "f1", "AUC-ROC","AP/PR-AUC"])
    
    # Todo lo que nos aporta información sobre la predicción
    best_models = []
    best_hyperparams = []
    matrices_confusion = []
    y_pred_list = []
    y_pred_proba_list = []
    
    imbModels = ImbGetBasedModelDef()
    
    i = 0
    for model_name, model in imbModels:
        for cjto_name, x_train, y_train, x_test, y_test in conjuntos:

            grid_search = GridSearchCV(model, scoring=scoring, n_jobs= -1, verbose = 1, param_grid=grids[i], cv=5)
            grid_search.fit(x_train, y_train)
            
            best_model = grid_search.best_estimator_
            best_models.append([model_name+"-"+cjto_name, best_model])
            
            best_params = grid_search.best_params_
            best_hyperparams.append([model_name+"-"+cjto_name, best_params])

            y_pred = best_model.predict(x_test)
            y_pred_list.append([model_name+"-"+cjto_name, y_pred])
            
            y_pred_proba = best_model.predict_proba(x_test)[:,-1]
            y_pred_proba_list.append([model_name+"-"+cjto_name, y_pred_proba])

            print(model_name + " " + cjto_name)
            print()
            
            cm =metrics.confusion_matrix(y_test, y_pred)
            print(cm)
            
            matrices_confusion.append([model_name+"-"+cjto_name, cm])
            print(metrics.classification_report(y_test,y_pred))

            roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
            print("ROC-AUC", round(roc_auc, 4))
            
            average_precision = metrics.average_precision_score(y_test, y_pred_proba)
            print("AP/PR-AUC: ",round(average_precision,4))

            # Calcular las métricas de rendimiento
            accuracy = metrics.accuracy_score(y_test, y_pred)
            precision = metrics.precision_score(y_test, y_pred)
            recall = metrics.recall_score(y_test, y_pred)
            f1 = metrics.f1_score(y_test, y_pred)
            resultados.loc[len(resultados.index)] = [model_name, cjto_name, accuracy, precision, recall, f1, roc_auc, average_precision]

        i+=1
        
    return [resultados, best_models, best_hyperparams, matrices_confusion, y_pred_list, y_pred_proba_list]

In [ ]:
# Función auxiliar que nos ayuda a encontrar el mejor modelo de la lista de modelos devuelta por el proceso de la función bucle_proceso_general

# nombre_buscado es el nombre de la mejor combinacion de preprocesado y algoritmo que buscamos en la
# lista de mejores modelos para cada combinación
def busca_tupla_nombre(nombre_buscado, best_models):
    tupla = None
    for nombre, params in best_hyperparams:
        if nombre == nombre_buscado:
            tupla = (nombre, params)
            break
    return tupla

In [ ]:
# Función que devuelve el mejor resultado de la lista de resultados de la función bucle_proceso_general

# resultados es el dataframe con todos los resultados de cada combinación de preprocesado y algoritmo
# scoring es la columna que comparamos para sacar el mejor
def mejor_resultado(resultados, scoring):
    idx_max = resultados[scoring].idxmax()
    fila_max = resultados.loc[idx_max]
    print(fila_max)
    nombre_buscado = fila_max['modelo']+"-"+fila_max['conjunto']
    print(nombre_buscado)
    return [fila_max, nombre_buscado]

In [ ]:
# Función que guarda de la lista de elementos obtenidos por la función bucle_proceso_general, aquel que nos haya dado los mejores resultados


def save_best(lista, file, nombre_buscado):
    tupla = None
    for nombre, elemento in lista:
        if nombre == nombre_buscado:
            tupla = (nombre, elemento)
            break

    joblib.dump(tupla, file)
    return tupla

In [ ]:
def find_element(nombre, lista):
    for nombre_element, element in lista:
        if nombre_element == nombre:
            element_return = element
            break
    return element_return

In [ ]:
def find_preproc(nombre, conjunto):
    for nombre_aux, train_x, train_y, test_x, test_y in conjunto:
        print(nombre_aux)
        if nombre == nombre_aux:
            tupla = [nombre_aux, train_x, train_y, test_x, test_y]
            break
    return tupla

In [ ]:
# Función que devuelve las medidas de desempeño a partir de un modelo Dummy, que depende del parámetros strategy

from sklearn.dummy import DummyClassifier

def DummyResults(train_x, train_y, test_x, test_y, strategy="most_frequent"):
    # Crear el modelo dummy
    dummy_clf = DummyClassifier(strategy=strategy)

    dummy_clf.fit(train_x, train_y)

    # Hacer predicciones con el modelo dummy
    y_pred_dummy = dummy_clf.predict(test_x)
    y_pred_proba_dummy = dummy_clf.predict_proba(test_x)[:,-1]
    cm =metrics.confusion_matrix(test_y, y_pred_dummy)
    print(cm)

    print(metrics.classification_report(test_y,y_pred_dummy))
    
    # Calcular métricas de rendimiento
    f1 = metrics.f1_score(test_y, y_pred_dummy)
    accuracy = metrics.accuracy_score(test_y, y_pred_dummy)
    precision = metrics.precision_score(test_y, y_pred_dummy)
    recall = metrics.recall_score(test_y, y_pred_dummy)

    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

    roc_auc = metrics.roc_auc_score(test_y, y_pred_proba_dummy)
    print("ROC-AUC", round(roc_auc, 4))
    
    average_precision = metrics.average_precision_score(test_y, y_pred_proba_dummy)
    print("AP/PR-AUC: ",round(average_precision,4))

Para obtener el dataset con los datos para la replicación de los experimentos, consultar el README del repositorio.

In [ ]:
DATASET_FLICKR =
DATASET_IG = 
DATASET_FLICKR_IG =

Grid de hiperparámetros a utilizar

In [ ]:
grids = [
    {# LDA
        'solver' : ['svd', 'lsqr'],
        'n_components' : [None] + [1, 2, 5, 8, 13, 21, 34, 55],
        'store_covariance' : [True, False],
        'tol' : [1e-2, 1e-3, 1e-5, 1e-7, 1e-9, 1e-11]
    },
    {# SVM
        'C': [1,10,100,1000],
        'gamma': [1,0.1,0.001,0.0001], 
        'kernel': ['linear','rbf']
    },
    {# RF
        'bootstrap': [True, False],
         'max_depth': [10, 50, 100, None],
         'max_features': ['auto', 'sqrt'],
        'criterion' :['gini', 'entropy'],
         'n_estimators': [200, 400, 600, 800]
    },
    {# GBM
        'n_estimators': [100, 200, 500],  # Número de árboles
        'learning_rate': [0.01, 0.1],  # Tasa de aprendizaje
        'max_depth': [3, 5, 7],  # Profundidad máxima de los árboles
        'max_features': ['auto', 'sqrt', 'log2']  # Fracción de características consideradas para dividir en cada nodo

    }
]

# Flickr

In [ ]:
df_flickr = pd.read_csv(DATASET_FLICKR)
train_flickr_x, train_flickr_y, test_flickr_x, test_flickr_y = data_partition(df_flickr, 0.3, 'is_photographer')

In [ ]:
conjuntos_flickr = conjuntos_preprocesamiento(train_flickr_x, test_flickr_x, train_flickr_y, test_flickr_y,
                              with_smote=True)
len(conjuntos_flickr)

In [ ]:
import os

if (not os.path.exists(f"resultados_flickr.csv")):
    resultados_flickr, best_models_flickr, best_hyperparams_flickr, matrices_confusion_flickr, y_pred_list_flickr, y_pred_proba_list_flickr = bucle_proceso_general(conjuntos_flickr,  grids, scoring = "average_precision")
    resultados_flickr.to_csv("resultados_flickr.csv", index = False)
    
    fila_flickr, nombre_flickr = mejor_resultado(resultados_flickr, "AP/PR-AUC")
    
    best_model_flickr = save_best(best_models_flickr, 'mejor_modelo_flickr.joblib', nombre_flickr)
    best_params_flickr = save_best(best_hyperparams_flickr, 'mejor_params_flickr.joblib', nombre_flickr)
    best_matrix_flickr = save_best(matrices_confusion_flickr, 'mejor_matrix_flickr.joblib', nombre_flickr)
    best_ypred_flickr = save_best(y_pred_list_flickr, 'mejor_ypred_flickr.joblib', nombre_flickr)
    best_ypred_proba_flickr = save_best(y_pred_proba_list_flickr, 'mejor_ypred_proba_flickr.joblib', nombre_flickr)
else:
    resultados_flickr = pd.read_csv("resultados_flickr.csv")
    
    fila_flickr, nombre_flickr = mejor_resultado(resultados_flickr, "AP/PR-AUC")
    
    best_model_flickr = joblib.load('mejor_modelo_flickr.joblib')
    best_params_flickr = joblib.load('mejor_params_flickr.joblib')
    best_matrix_flickr = joblib.load('mejor_matrix_flickr.joblib')
    best_ypred_flickr = joblib.load('mejor_ypred_flickr.joblib')
    best_ypred_proba_flickr = joblib.load('mejor_ypred_proba_flickr.joblib')

In [ ]:
resultados_flickr

## Desempate

Hemos obtenido dos modelos con rendimientos muy parecidos en el proceso anterior, analizando las otras métricas, nos quedamos con aquel que mejor F1 ha obtenido, es decir, el que aplica algoritmo LDA con el preprocesado QT+SMOTE

In [ ]:
for nombre, train_aux_x, train_aux_y,test_aux_x, test_aux_y in conjuntos_flickr:
    if nombre == "qt_smote":
        tupla_aux = [nombre, train_aux_x, train_aux_y,test_aux_x, test_aux_y]
        break
tupla_aux[0]

In [ ]:
for nombre, model in best_models_flickr:
    if nombre == "LDA-qt_smote":
        model_aux = model
        break
model_aux

In [ ]:
filas_aux = [0.774, 0.545, 0.75, 0.632, 0.845, 0.644]

In [ ]:
filas_aux = resultados_flickr.loc[(resultados_flickr['modelo'] == "LDA") & (resultados_flickr['conjunto'] == "qt_smote")]
filas_aux = filas_aux.values.tolist()
filas_aux = [item for sublist in filas_aux for item in sublist]
filas_aux

In [ ]:
show_bars_metrics(filas_aux[2:8], "Medidas de Desempeño para Caso Flickr")

In [ ]:
for nombre, y_pred in y_pred_list_flickr:
    if nombre == "LDA-qt_smote":
        y_pred_aux = y_pred
        break
y_pred_aux

In [ ]:
for nombre, y_pred_proba in y_pred_proba_list_flickr:
    if nombre == "LDA-qt_smote":
        y_pred_proba_aux = y_pred_proba
        break
y_pred_proba_aux

In [ ]:
matrix_confusion_pred(test_flickr_y, y_pred_aux)

In [ ]:
threshold_flickr = pr_auc(test_flickr_y, y_pred_proba_aux)

## Continuación

In [ ]:
fila_flickr

In [ ]:
show_bars_metrics(fila_flickr.values[2:8], "Medidas de Desempeño para Caso Flickr")

In [ ]:
matrix_confusion_pred(test_flickr_y, best_ypred_flickr[1])

In [ ]:
y_pred_proba_flickr = best_ypred_proba_flickr[1]

In [ ]:
roc_auc(test_flickr_y, y_pred_proba_flickr)

In [ ]:
threshold_flickr = pr_auc(test_flickr_y, y_pred_proba_flickr)

In [ ]:
# Convertir probabilidades a etiquetas binarias usando el umbral
y_pred_threshold = (y_pred_proba_flickr >= threshold_flickr).astype(int)

# Calcular métricas de rendimiento
f1 = metrics.f1_score(test_flickr_y, y_pred_threshold)
accuracy = metrics.accuracy_score(test_flickr_y, y_pred_threshold)
precision = metrics.precision_score(test_flickr_y, y_pred_threshold)
recall = metrics.recall_score(test_flickr_y, y_pred_threshold)

cm =metrics.confusion_matrix(test_flickr_y, y_pred_threshold)
print(cm)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Instagram

In [ ]:
df_ig = pd.read_csv(DATASET_IG)
train_ig_x, train_ig_y, test_ig_x, test_ig_y = data_partition(df_ig, 0.3, 'is_photographer')

In [ ]:
conjuntos_ig = conjuntos_preprocesamiento(train_ig_x, test_ig_x, train_ig_y, test_ig_y,
                              with_smote=True)
len(conjuntos_ig)

In [ ]:
import os

if (not os.path.exists(f"resultados_ig.csv")):
    resultados_ig, best_models_ig, best_hyperparams_ig, matrices_confusion_ig, y_pred_list_ig, y_pred_proba_list_ig = bucle_proceso_general(conjuntos_ig,  grids, scoring = "average_precision")
    resultados_ig.to_csv("resultados_ig.csv", index = False)
    
    fila_ig, nombre_ig = mejor_resultado(resultados_ig, "AP/PR-AUC")
    
    best_model_ig = save_best(best_models_ig, 'mejor_modelo_ig.joblib', nombre_ig)
    best_params_ig = save_best(best_hyperparams_ig, 'mejor_params_ig.joblib', nombre_ig)
    best_matrix_ig = save_best(matrices_confusion_ig, 'mejor_matrix_ig.joblib', nombre_ig)
    best_ypred_ig= save_best(y_pred_list_ig, 'mejor_ypred_ig.joblib', nombre_ig)
    best_ypred_proba_ig = save_best(y_pred_proba_list_ig, 'mejor_ypred_proba_ig.joblib', nombre_ig)
else:
    resultados_ig = pd.read_csv("resultados_ig.csv")
    
    fila_ig, nombre_ig = mejor_resultado(resultados_ig, "AP/PR-AUC")
    
    best_model_ig = joblib.load('mejor_modelo_ig.joblib')
    best_params_ig = joblib.load('mejor_params_ig.joblib')
    best_matrix_ig = joblib.load('mejor_matrix_ig.joblib')
    best_ypred_ig = joblib.load('mejor_ypred_ig.joblib')
    best_ypred_proba_ig= joblib.load('mejor_ypred_proba_ig.joblib')

In [ ]:
resultados_ig

In [ ]:
fila_ig

In [ ]:
show_bars_metrics(fila_ig.values[2:8], "Medidas de Desempeño para Caso Instagram")

In [ ]:
matrix_confusion_pred(test_ig_y, best_ypred_ig[1])

In [ ]:
y_pred_proba_ig = best_ypred_proba_ig[1]

In [ ]:
roc_auc(test_ig_y, y_pred_proba_ig)

In [ ]:
threshold_ig = pr_auc(test_ig_y, y_pred_proba_ig)

In [ ]:
# Convertir probabilidades a etiquetas binarias usando el umbral
y_pred_threshold = (y_pred_proba_ig >= threshold_ig).astype(int)

# Calcular métricas de rendimiento
f1 = metrics.f1_score(test_ig_y, y_pred_threshold)
accuracy = metrics.accuracy_score(test_ig_y, y_pred_threshold)
precision = metrics.precision_score(test_ig_y, y_pred_threshold)
recall = metrics.recall_score(test_ig_y, y_pred_threshold)

cm =metrics.confusion_matrix(test_ig_y, y_pred_threshold)
print(cm)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Flickr + Instagram

In [ ]:
df_igflickr = pd.read_csv(DATASET_FLICK_IG)
train_igflickr_x, train_igflickr_y, test_igflickr_x, test_igflickr_y = data_partition(df_igflickr, 0.3, 'is_photographer')

In [ ]:
conjuntos_igflickr = conjuntos_preprocesamiento(train_igflickr_x, test_igflickr_x, train_igflickr_y, test_igflickr_y,
                              with_smote=True)
len(conjuntos_igflickr)

In [ ]:
import os

if (not os.path.exists(f"resultados_igflickr.csv")):
    resultados_igflickr, best_models_igflickr, best_hyperparams_igflickr, matrices_confusion_igflickr, y_pred_list_igflickr, y_pred_proba_list_igflickr = bucle_proceso_general(conjuntos_igflickr,  grids, scoring = "average_precision")
    resultados_igflickr.to_csv("resultados_igflickr.csv", index = False)
    
    fila_igflickr, nombre_igflickr = mejor_resultado(resultados_igflickr, "AP/PR-AUC")
    
    best_model_igflickr = save_best(best_models_igflickr, 'mejor_modelo_igflickr.joblib', nombre_igflickr)
    best_params_igflickr = save_best(best_hyperparams_igflickr, 'mejor_params_igflickr.joblib', nombre_igflickr)
    best_matrix_igflickr = save_best(matrices_confusion_igflickr, 'mejor_matrix_igflickr.joblib', nombre_igflickr)
    best_ypred_igflickr = save_best(y_pred_list_igflickr, 'mejor_ypred_igflickr.joblib', nombre_igflickr)
    best_ypred_proba_igflickr = save_best(y_pred_proba_list_igflickr, 'mejor_ypred_proba_igflickr.joblib', nombre_igflickr)
else:
    resultados_igflickr = pd.read_csv("resultados_igflickr.csv")
    
    fila_igflickr, nombre_igflickr = mejor_resultado(resultados_igflickr, "AP/PR-AUC")
    
    best_model_igflickr = joblib.load('mejor_modelo_igflickr.joblib')
    best_params_igflickr = joblib.load('mejor_params_igflickr.joblib')
    best_matrix_igflickr = joblib.load('mejor_matrix_igflickr.joblib')
    best_ypred_igflickr = joblib.load('mejor_ypred_igflickr.joblib')
    best_ypred_proba_igflickr = joblib.load('mejor_ypred_proba_igflickr.joblib')
    

In [ ]:
resultados_igflickr

In [ ]:
fila_igflickr

In [ ]:
show_bars_metrics(fila_igflickr.values[2:8], "Caso Flickr+Instagram")

In [ ]:
matrix_confusion_pred(test_igflickr_y, best_ypred_igflickr[1])

In [ ]:
y_pred_proba_igflickr = best_ypred_proba_igflickr[1]

In [ ]:
roc_auc(test_igflickr_y, y_pred_proba_igflickr)

In [ ]:
threshold_igflickr = pr_auc(test_igflickr_y, y_pred_proba_igflickr)

In [ ]:
# Convertir probabilidades a etiquetas binarias usando el umbral
y_pred_threshold = (y_pred_proba_igflickr >= threshold_igflickr).astype(int)

# Calcular métricas de rendimiento
f1 = metrics.f1_score(test_igflickr_y, y_pred_threshold)
accuracy = metrics.accuracy_score(test_igflickr_y, y_pred_threshold)
precision = metrics.precision_score(test_igflickr_y, y_pred_threshold)
recall = metrics.recall_score(test_igflickr_y, y_pred_threshold)

cm =metrics.confusion_matrix(test_igflickr_y, y_pred_threshold)
print(cm)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Modelo Instagram con Datos Flickr

In [ ]:
fila_ig

In [ ]:
nombre_ig

In [ ]:
flickr_x = df_flickr.loc[:, df_flickr.columns != 'is_photographer']
flickr_y = df_flickr.loc[:, 'is_photographer']

scaler = StandardScaler()
scaler.fit(flickr_x)
flickrScaled_x = pd.DataFrame(scaler.transform(flickr_x), columns=flickr_x.columns, index=flickr_x.index)

In [ ]:
y_pred1 = best_model_ig[1].predict(flickrScaled_x)
cm1 = confusion_matrix(flickr_y, y_pred1)
cm1

In [ ]:
matrix_confusion_pred(flickr_y, y_pred1)

In [ ]:
y_pred_proba1 = best_model_ig[1].predict_proba(flickr_x)[:,-1]
roc_auc1 = metrics.roc_auc_score(flickr_y, y_pred_proba1)
print("ROC-AUC", round(roc_auc1, 4))
            
average_precision1 = metrics.average_precision_score(flickr_y, y_pred_proba1)
print("AP/PR-AUC: ",round(average_precision1,4))

# Calcular las métricas de rendimiento
accuracy = metrics.accuracy_score(flickr_y, y_pred1)
precision = metrics.precision_score(flickr_y, y_pred1)
recall = metrics.recall_score(flickr_y, y_pred1)
f1 = metrics.f1_score(flickr_y, y_pred1)

In [ ]:
show_bars_metrics([accuracy, precision, recall, f1, roc_auc1, average_precision1], "Datos Flickr Modelo Ig")

In [ ]:
roc_auc(flickr_y, y_pred_proba1)

In [ ]:
threshold1 = pr_auc(flickr_y, y_pred_proba1)

In [ ]:
# Para obtener los datos de Flickr preprocesados según el modelo de IG
tupla_1 = find_preproc(nombre_ig.split('-')[1], conjuntos_flickr)

In [ ]:
y_pred1 = best_model_ig[1].predict(tupla_1[3])
cm1 = confusion_matrix(tupla_1[4], y_pred1)
cm1

In [ ]:
best_model_ig[1]

In [ ]:
y_pred_proba1 = best_model_ig[1].predict_proba(tupla_1[3])[:,-1]
roc_auc1 = metrics.roc_auc_score(tupla_1[4], y_pred_proba1)
print("ROC-AUC", round(roc_auc1, 4))
            
average_precision1 = metrics.average_precision_score(tupla_1[4], y_pred_proba1)
print("AP/PR-AUC: ",round(average_precision1,4))

# Calcular las métricas de rendimiento
accuracy = metrics.accuracy_score(tupla_1[4], y_pred1)
precision = metrics.precision_score(tupla_1[4], y_pred1)
recall = metrics.recall_score(tupla_1[4], y_pred1)
f1 = metrics.f1_score(tupla_1[4], y_pred1)

In [ ]:
show_bars_metrics([accuracy, precision, recall, f1, roc_auc1, average_precision1], "Datos Flickr Modelo Ig")

# Modelo Flickr con Datos Instagram

In [ ]:
fila_flickr

In [ ]:
nombre_flickr

In [ ]:
# Para obtener los datos de IG preprocesados según el modelo de Flickr
tupla_2 = find_preproc(nombre_flickr.split('-')[1], conjuntos_ig)

In [ ]:
for nombre, model in best_models_flickr:
    if nombre == "LDA-qt_smote":
        model_aux = model
        break
model_aux

In [ ]:
ig_x = df_ig.loc[:, df_flickr.columns != 'is_photographer']
ig_y = df_ig.loc[:, 'is_photographer']

scaler = StandardScaler()
scaler.fit(ig_x)
igScaled_x = pd.DataFrame(scaler.transform(ig_x), columns=ig_x.columns, index=ig_x.index)

In [ ]:
y_pred2 = model_aux.predict(igScaled_x)
cm1 = confusion_matrix(ig_y, y_pred2)
cm1

In [ ]:
matrix_confusion_pred(ig_y, y_pred2)

In [ ]:
roc_auc

In [ ]:
y_pred_proba2 = model_aux.predict_proba(ig_x)[:,-1]
roc_auc2 = metrics.roc_auc_score(ig_y, y_pred_proba2)
print("ROC-AUC", round(roc_auc2, 4))
            
average_precision2 = metrics.average_precision_score(ig_y, y_pred_proba2)
print("AP/PR-AUC: ",round(average_precision2,4))

# Calcular las métricas de rendimiento
accuracy = metrics.accuracy_score(ig_y, y_pred2)
precision = metrics.precision_score(ig_y, y_pred2)
recall = metrics.recall_score(ig_y, y_pred2)
f1 = metrics.f1_score(ig_y, y_pred2)

In [ ]:
show_bars_metrics([accuracy, precision, recall, f1, roc_auc2, average_precision2], "Datos Ig Modelo Flickr")

In [ ]:
del(roc_auc)

In [ ]:
roc_auc(ig_y, y_pred_proba2)

In [ ]:
threshold2 = pr_auc(ig_y, y_pred_proba2)

In [ ]:
best_model_flickr[1]

In [ ]:
y_pred2 = best_model_ig[1].predict(tupla_2[3])
cm2 = confusion_matrix(tupla_2[4], y_pred2)
cm2

In [ ]:
y_pred_proba2 = best_model_flickr[1].predict_proba(tupla_2[3])[:,-1]
roc_auc2 = metrics.roc_auc_score(tupla_2[4], y_pred_proba2)
print("ROC-AUC", round(roc_auc2, 4))
            
average_precision2 = metrics.average_precision_score(tupla_2[4], y_pred_proba2)
print("AP/PR-AUC: ",round(average_precision2,4))

# Calcular las métricas de rendimiento
accuracy = metrics.accuracy_score(tupla_2[4], y_pred2)
precision = metrics.precision_score(tupla_2[4], y_pred2)
recall = metrics.recall_score(tupla_2[4], y_pred2)
f1 = metrics.f1_score(tupla_2[4], y_pred2)

In [ ]:
show_bars_metrics([accuracy, precision, recall, f1, roc_auc2, average_precision2], "Medidas de Desempeño para Datos Ig Modelo Flickr")